# Standardize Data - Generalized for Years

Checklist  
- [x] Gene Expression (RNAseq)
- [x] Cell Percentages (CyTOF)
- [x] Ab Titers
- [x] Protein levels (Olink)
- [x] Task Vectors
- [x] Metadata

In [1]:
import os 
import pandas as pd 
import numpy as np
os.chdir('C:/Users/jreyna/Documents/Projects/cmi-pb-multiomics/third_challenge')
outdir = 'results/main/cmi_pb_datasets/processed/all_versus_all/'
os.makedirs(outdir, exist_ok=True)

# create a harmonized directory
harmony_dir = os.path.join(outdir, 'harmonized/')
os.makedirs(harmony_dir, exist_ok=True)

IgG1 and IgG4 day 14 and day 0 values for PT, FHA, and Pertactin

In [2]:
# setting a list of assays
assays = ('plasma_cytokine_concentration', 'pbmc_cell_frequency', 'plasma_ab_titer', 'pbmc_gene_expression')

In [3]:
# loading gencode data
gencode = pd.read_table('../shared/refs/gencode/gencode.v38lift37.annotation.protein_coding.bed',
                        header=None)
gencode.columns = ['chr', 'start', 'end', 'strand', 'gene_id', 'gene_name']
gencode['gene_nonversioned_id'] = gencode['gene_id'].str.replace('\..*', '', regex=True)

In [4]:
interm_dir = os.path.join(outdir, 'full')
os.makedirs(interm_dir, exist_ok=True)

## Load the datasets

In [5]:
years = [2020, 2021, 2022]
datasets = {2020: {}, 2021: {}, 2022: {}}
ld_or_bds = {2020: 'LD', 2021: 'LD', 2022: 'BD'}

#### Parse and process the metadata

In [6]:
for year in years:
    
    # get ld_or_bd
    ld_or_bd = ld_or_bds[year]
    
    # get specimen and sample table
    for metatable in ('subject', 'specimen'): 
        params = {'year': year, 'ld_or_bd': ld_or_bd, 'mtype': metatable}
        fn = 'results/main/cmi_pb_datasets/raw/{year}{ld_or_bd}_{mtype}.tsv'.format(**params)
        df = pd.read_table(fn)
        datasets[year][metatable] = df
    
    # get meta master table
    subjects = datasets[year]['subject']
    specimen = datasets[year]['specimen']
    master_meta = subjects.merge(specimen, on='subject_id')
    master_meta = master_meta[['subject_id',
                               'specimen_id',
                               'infancy_vac',
                               'biological_sex',
                               'year_of_birth',
                               'date_of_boost',
                               'actual_day_relative_to_boost',
                               'planned_day_relative_to_boost',
                               'ethnicity',
                               'race',
                               'dataset',
                               'specimen_type',
                               'visit']]

    datasets[year]['master_meta'] = master_meta

    # save meta
    meta_fn = os.path.join(interm_dir, 'meta.{year}.pivoted.tsv'.format(**params))
    datasets[year]['master_meta'].to_csv(meta_fn, sep='\t')

#### Parse and process the assay data

In [7]:
# code to switch between different MFI's 
curr_mfi, curr_mfi_name = ('MFI', 'mfi_raw')

In [8]:
# defining the meta columns (used to remove columns later one)
meta_cols = ['specimen_id', 'infancy_vac', 'biological_sex',
                'year_of_birth', 'date_of_boost', 'actual_day_relative_to_boost',
                'ethnicity', 'race', 'dataset',
                'specimen_type', 'visit']


In [9]:
for year in years:

    print('year:', year)
    
    # get ld_or_bd
    ld_or_bd = ld_or_bds[year]
    
    for i, assay in enumerate(assays): 
        
        print('\tassay:', assay)
        
        params = {'year': year, 'ld_or_bd': ld_or_bd, 'mtype': assay}
        fn = 'results/main/cmi_pb_datasets/raw/{year}{ld_or_bd}_{mtype}.tsv'.format(**params)

        print('\t\tfn:', fn) 

        df = pd.read_table(fn)
        
        master_meta = datasets[year]['master_meta']

        if assays[i] == 'pbmc_gene_expression':
            df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '', regex=True)
            df = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='tpm')
            df = master_meta.merge(df, on='specimen_id')

        elif assays[i] == 'pbmc_cell_frequency':
            df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
            df = master_meta.merge(df, on='specimen_id')

        elif assays[i] == 'plasma_cytokine_concentration':
            #df = df[df['quality_control'] == 'Pass']
            df = df[df['protein_expression'] > df['lower_limit_of_quantitation']]
            df = df.pivot(index='specimen_id', columns='protein_id', values='protein_expression')
            df = master_meta.merge(df, on='specimen_id')      

        elif assays[i] == 'plasma_ab_titer':
            aglist = ['1% PFA PT', 'PT', 'PRN', 'FHA', 'FIM2/3']
            df = df.loc[df.antigen.isin(aglist), :]
            df['isotype_antigen'] = df['isotype'] + '-' + df['antigen']
            df = df.pivot(index='specimen_id', columns='isotype_antigen', values=curr_mfi)
            df = master_meta.merge(df, on='specimen_id')  

        datasets[year][assay] = df.drop(meta_cols, axis=1)  

year: 2020
	assay: plasma_cytokine_concentration
		fn: results/main/cmi_pb_datasets/raw/2020LD_plasma_cytokine_concentration.tsv
	assay: pbmc_cell_frequency
		fn: results/main/cmi_pb_datasets/raw/2020LD_pbmc_cell_frequency.tsv
	assay: plasma_ab_titer
		fn: results/main/cmi_pb_datasets/raw/2020LD_plasma_ab_titer.tsv
	assay: pbmc_gene_expression
		fn: results/main/cmi_pb_datasets/raw/2020LD_pbmc_gene_expression.tsv
year: 2021
	assay: plasma_cytokine_concentration
		fn: results/main/cmi_pb_datasets/raw/2021LD_plasma_cytokine_concentration.tsv
	assay: pbmc_cell_frequency
		fn: results/main/cmi_pb_datasets/raw/2021LD_pbmc_cell_frequency.tsv
	assay: plasma_ab_titer
		fn: results/main/cmi_pb_datasets/raw/2021LD_plasma_ab_titer.tsv
	assay: pbmc_gene_expression
		fn: results/main/cmi_pb_datasets/raw/2021LD_pbmc_gene_expression.tsv
year: 2022
	assay: plasma_cytokine_concentration
		fn: results/main/cmi_pb_datasets/raw/2022BD_plasma_cytokine_concentration.tsv
	assay: pbmc_cell_frequency
		fn: res

In [10]:
for year in years:
    for assay, df in datasets[year].items():
        print(year, assay, datasets[year][assay].subject_id.nunique())
    print()

2020 subject 60
2020 specimen 60
2020 master_meta 60
2020 plasma_cytokine_concentration 18
2020 pbmc_cell_frequency 20
2020 plasma_ab_titer 58
2020 pbmc_gene_expression 36

2021 subject 36
2021 specimen 36
2021 master_meta 36
2021 plasma_cytokine_concentration 36
2021 pbmc_cell_frequency 33
2021 plasma_ab_titer 33
2021 pbmc_gene_expression 36

2022 subject 22
2022 specimen 21
2022 master_meta 21
2022 plasma_cytokine_concentration 20
2022 pbmc_cell_frequency 21
2022 plasma_ab_titer 20
2022 pbmc_gene_expression 21



In [37]:
for year in years:
    print('year:', year)

    # save a dataframe for each assay on each day
    for assay, df in datasets[year].items():
        
        if assay in assays:
            
            print('\tassay:', assay)
            
            for day, day_df in df.groupby('planned_day_relative_to_boost'):

                if assay == 'ab_titer': 
                    outfn = os.path.join(interm_dir, '{}.{}.{}.day{}.pivoted.tsv'.format(assay, year, curr_mfi_name, day))
                else:
                    outfn = os.path.join(interm_dir, '{}.{}.day{}.pivoted.tsv'.format(assay, year, day))

                day_df.drop('planned_day_relative_to_boost', axis=1).to_csv(outfn, index=False, sep='\t')
                print('\t\tday:', day, day_df.shape)

year: 2020
	assay: plasma_cytokine_concentration
		day: 0 (18, 257)
		day: 1 (18, 257)
		day: 3 (18, 257)
		day: 7 (18, 257)
		day: 14 (18, 257)
	assay: pbmc_cell_frequency
		day: 0 (20, 27)
		day: 1 (20, 27)
		day: 3 (20, 27)
		day: 7 (20, 27)
		day: 14 (20, 27)
	assay: plasma_ab_titer
		day: 0 (58, 25)
		day: 1 (47, 25)
		day: 3 (56, 25)
		day: 7 (56, 25)
		day: 14 (57, 25)
		day: 30 (54, 25)
		day: 120 (52, 25)
		day: 366 (1, 25)
		day: 386 (1, 25)
		day: 396 (1, 25)
		day: 402 (1, 25)
		day: 407 (1, 25)
		day: 415 (1, 25)
		day: 423 (1, 25)
		day: 428 (1, 25)
		day: 447 (1, 25)
		day: 448 (1, 25)
		day: 464 (2, 25)
		day: 511 (1, 25)
		day: 543 (1, 25)
	assay: pbmc_gene_expression
		day: 0 (36, 58304)
		day: 1 (36, 58304)
		day: 3 (36, 58304)
		day: 7 (36, 58304)
		day: 14 (36, 58304)
year: 2021
	assay: plasma_cytokine_concentration
		day: 0 (36, 45)
		day: 1 (36, 45)
		day: 3 (36, 45)
		day: 7 (36, 45)
		day: 14 (36, 45)
	assay: pbmc_cell_frequency
		day: 0 (33, 52)
		day: 1 (33, 

## Standardize RNA-seq

The RNA-seq dataset includes a lot more variables and processing so there are many steps we are including for standardization:

    1) remove zero variance genes,
    2) remove mitochondrial genes, 
    3) keep expressed genes (defined as those with TPM > 1) in more than a certain proportion of people (cut_filter)
    4) intersect genes across all years and harmonize all tables 

In [12]:
# load gene data to find mitochondrial genes
mito_genes = gencode.loc[(gencode.chr == 'chrM'), 'gene_nonversioned_id']
mito_genes = set(mito_genes.unique().tolist())

In [13]:
# value used across both RNA-seq datasets
cut_filter = 0.3

In [14]:
for year in datasets.keys():
    
    # keep the full dt
    full_df = datasets[year]['pbmc_gene_expression']
    
    # find zero variance genes 
    tmp = full_df.loc[full_df.planned_day_relative_to_boost.isin([0, -15, -30]),:]
    tmp = tmp.loc[:, tmp.columns.str.match('ENSG')]
    tmpvars = tmp.var()
    zero_var_genes = tmpvars[(tmpvars == 0)]
    zero_var_genes = set(zero_var_genes.index.tolist())

    # locate genes which are expressed
    # defined as genes with TPM > 1 in more than 30% of samples 
    expressed_genes = (tmp > 1).sum() > cut_filter * tmp.shape[0]
    expressed_genes = set(expressed_genes.index[expressed_genes.values.tolist()])

    # get the final list of genes 
    keep_genes = expressed_genes.difference(mito_genes).difference(zero_var_genes)
    keep_genes = keep_genes.intersection(gencode.gene_nonversioned_id.values.tolist())

    # make the final new_rnaseq table
    new_rnaseq1 = full_df.loc[:, ~full_df.columns.str.match('ENSG')]
    new_rnaseq2 = full_df.loc[:, list(keep_genes)]
    datasets[year]['pbmc_gene_expression_std'] = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
    

### Harmonize the RNA-seq datasets 

In [15]:
# getting the shared genes accross all years
shared_genes = set(datasets[years[0]]['pbmc_gene_expression_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
for year in years[1:]:
    curr_genes =  set(datasets[year]['pbmc_gene_expression_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
    shared_genes = shared_genes.intersection(curr_genes)
shared_genes = sorted(shared_genes)

In [16]:
# getting harmonized for all years
for year in years:
    tdf = datasets[year]['pbmc_gene_expression_std']
    new_rnaseq1 = tdf.loc[:, ~tdf.columns.str.match('ENSG')]
    new_rnaseq2 = tdf.loc[:, shared_genes]
    harmonized_df = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
    datasets[year]['pbmc_gene_expression_harmonized'] = harmonized_df

In [17]:
# save harmonized rnaseq
# getting harmonized for all years
for year in years:
    harmonized_fn = os.path.join(harmony_dir, f'pbmc_gene_expression.{year}.pivoted.tsv')
    datasets[year]['pbmc_gene_expression_harmonized'].to_csv(harmonized_fn, index=False, sep='\t')

## Standardize CyTOF

In [18]:
# getting the shared cells accross all years
shared_cells = set(datasets[years[0]]['pbmc_cell_frequency'].columns.tolist()[2:])
for year in years[1:]:
    curr_cells = set(datasets[year]['pbmc_cell_frequency'].columns.tolist()[2:])
    shared_cells = shared_cells.intersection(curr_cells)
shared_cells = sorted(shared_cells)

In [19]:
# getting harmonized for all years
for year in years:
    tdf = datasets[year]['pbmc_cell_frequency']
    new_cells1 = tdf.loc[:, tdf.columns.tolist()[0:2]]
    new_cells2 = tdf.loc[:, shared_cells]
    harmonized_df = pd.concat([new_cells1, new_cells2], axis=1)
    datasets[year]['pbmc_cell_frequency_harmonized'] = harmonized_df

In [20]:
# getting harmonized for all years
for year in years:
    harmonized_fn = os.path.join(harmony_dir, f'pbmc_cell_frequency.{year}.pivoted.tsv')
    datasets[year]['pbmc_cell_frequency_harmonized'].to_csv(harmonized_fn, index=False, sep='\t')

## Standardize Ab Titers

In [21]:
# getting the shared proteins accross all years
shared_proteins = set(datasets[years[0]]['plasma_ab_titer'].columns.tolist()[2:])
for year in years[1:]:
    curr_proteins = set(datasets[year]['plasma_ab_titer'].columns.tolist()[2:])
    shared_proteins = shared_proteins.intersection(curr_proteins)
shared_proteins = sorted(shared_proteins)

In [22]:
# getting harmonized for all years
for year in years:
    tdf = datasets[year]['plasma_ab_titer']
    new_proteins1 = tdf.loc[:, tdf.columns.tolist()[0:2]]
    new_proteins2 = tdf.loc[:, shared_proteins]
    harmonized_df = pd.concat([new_proteins1, new_proteins2], axis=1)
    datasets[year]['plasma_ab_titer_harmonized'] = harmonized_df

In [23]:
# getting harmonized for all years
for year in years:
    harmonized_fn = os.path.join(harmony_dir, f'plasma_ab_titer.{curr_mfi_name}.{year}.pivoted.tsv')
    datasets[year]['plasma_ab_titer_harmonized'].to_csv(harmonized_fn, index=False, sep='\t')

## Standardize Cytokine Concentrations

In [24]:
# getting the shared proteins accross all years
shared_proteins = set(datasets[years[0]]['plasma_cytokine_concentration'].columns.tolist()[2:])
for year in years[1:]:
    curr_proteins = set(datasets[year]['plasma_cytokine_concentration'].columns.tolist()[2:])
    shared_proteins = shared_proteins.intersection(curr_proteins)
shared_proteins = sorted(shared_proteins)

In [25]:
# getting harmonized for all years
for year in years:
    tdf = datasets[year]['plasma_cytokine_concentration']
    new_proteins1 = tdf.loc[:, tdf.columns.tolist()[0:2]]
    new_proteins2 = tdf.loc[:, shared_proteins]
    harmonized_df = pd.concat([new_proteins1, new_proteins2], axis=1)
    datasets[year]['plasma_cytokine_concentration_harmonized'] = harmonized_df

In [26]:
# getting harmonized for all years
for year in years:
    harmonized_fn = os.path.join(harmony_dir, f'plasma_cytokine_concentration.{year}.pivoted.tsv')
    datasets[year]['plasma_cytokine_concentration_harmonized'].to_csv(harmonized_fn, index=False, sep='\t')

## Providing all task vectors for training

The tasks are described here: https://www.cmi-pb.org/blog/prediction-challenge-overview/#Prediction%20challenge%20tasks 

In [27]:
from IPython.display import display

In [28]:
# Loading the table describing each task 
tasks = pd.read_table('results/refs/tasks.tsv')

In [29]:
# Making a task vector using GENE IDS for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # assign the task type
    ttype = 'normal'
    if '/' in task.days:
        ttype = 'fold-change'
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    
    # process task requiring a single day "normal"
    if ttype == 'normal':
        tdata = []
        for year in years[:-1]:
            tdf = datasets[year]['{}_harmonized'.format(task.assay)]
            tdf = tdf.loc[tdf.planned_day_relative_to_boost == int(task.days), cols]
            tdata.append(tdf)
        task_data = pd.concat(tdata)
        
        # assign column names
        task_data.columns = ['subject_id', task.fetchname + '.day' + str(task.days)]   
        
    elif ttype == 'fold-change':
        tdata = []
        for year in years[:-1]:
            tdf = datasets[year]['{}_harmonized'.format(task.assay)]
            
            # parse days
            num_day, denom_day = task.days.split('/')
            num_day, denom_day = int(num_day), int(denom_day)
                       
            # extract data for each and calculate the fold change
            num_tdf = tdf.loc[tdf.planned_day_relative_to_boost == num_day, cols].\
                                set_index('subject_id').squeeze().sort_index()
            denom_tdf = tdf.loc[tdf.planned_day_relative_to_boost == denom_day, cols].\
                                set_index('subject_id').squeeze().sort_index()
            fc = num_tdf / denom_tdf

            tdf = fc.to_frame().reset_index()
            tdata.append(tdf)
        task_data = pd.concat(tdata)           
        
        # assign column names
        task_data.columns = ['subject_id', task.fetchname + '.fold-change-day' + str(task.days)]   

    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 
    
# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.feature_names.{}.tsv'.format(curr_mfi_name))
master_tasks.to_csv(task_fn, sep='\t', index=False)

In [30]:
master_tasks.head()

,subject_id,IgG-PT.day14,IgG-PT.fold-change-day14/0,Monocytes.day1,Monocytes.fold-change-day1/0,ENSG00000277632.day3,ENSG00000277632.fold-change-day3/0
0,1,199.517666,2.909857,NaN,NaN,46.410,1.180225
1,3,129.197956,6.422625,NaN,NaN,26.204,1.127005
2,4,144.885339,3.858236,7.211965,1.313454,13.353,0.830204
3,5,97.743258,1.402631,NaN,NaN,20.618,0.706048
4,6,167.496355,42.792746,41.380502,1.815489,19.606,0.858256


In [31]:
# Making a task vector using GENE NAMES for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # assign the task type
    ttype = 'normal'
    if '/' in task.days:
        ttype = 'fold-change'
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    
    # process task requiring a single day "normal"
    if ttype == 'normal':
        tdata = []
        for year in years[:-1]:
            tdf = datasets[year]['{}_harmonized'.format(task.assay)]
            tdf = tdf.loc[tdf.planned_day_relative_to_boost == int(task.days), cols]
            tdata.append(tdf)
        task_data = pd.concat(tdata)
        
        # assign column names, this time RNA-seq will use gene names
        if task.assay == 'rnaseq':
            task_data.columns = ['subject_id', task.fullname + '.day' + str(task.days)]  
        else:
            task_data.columns = ['subject_id', task.fetchname + '.day' + str(task.days)]
        
    elif ttype == 'fold-change':
        tdata = []
        for year in years[:-1]:
            tdf = datasets[year]['{}_harmonized'.format(task.assay)]
            
            # parse days
            num_day, denom_day = task.days.split('/')
            num_day, denom_day = int(num_day), int(denom_day)
                       
            # extract data for each and calculate the fold change
            num_tdf = tdf.loc[tdf.planned_day_relative_to_boost == num_day, cols].\
                                set_index('subject_id').squeeze().sort_index()
            denom_tdf = tdf.loc[tdf.planned_day_relative_to_boost == denom_day, cols].\
                                set_index('subject_id').squeeze().sort_index()
            fc = num_tdf / denom_tdf

            tdf = fc.to_frame().reset_index()
            tdata.append(tdf)
        task_data = pd.concat(tdata)           
        
        # assign column names, this time RNA-seq will use gene names
        if task.assay == 'rnaseq':
            task_data.columns = ['subject_id', task.fullname + '.fold-change-day' + str(task.days)]  
        else:
            task_data.columns = ['subject_id', task.fetchname + '.fold-change-day' + str(task.days)]   
        
    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 
    
# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.common_names.{}.tsv'.format(curr_mfi_name))
master_tasks.to_csv(task_fn, sep='\t', index=False)

## Providing meta data into a basic format

In [32]:
drop_cols = ['specimen_id', 'actual_day_relative_to_boost', 'planned_day_relative_to_boost', 'visit', 'dataset']
for year in years: 
    metadata = datasets[year]['master_meta'].drop_duplicates('subject_id')
    metadata.drop(drop_cols, axis=1, inplace=True)
    
    basic_fn = os.path.join(harmony_dir, 'clinical_metadata.{}.tsv'.format(year))
    metadata.to_csv(basic_fn, sep='\t', index=False)


C:\Users\jreyna\AppData\Local\Temp\ipykernel_25528\1287468092.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata.drop(drop_cols, axis=1, inplace=True)
C:\Users\jreyna\AppData\Local\Temp\ipykernel_25528\1287468092.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata.drop(drop_cols, axis=1, inplace=True)
C:\Users\jreyna\AppData\Local\Temp\ipykernel_25528\1287468092.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met

# Summarize the harmonization process

In [33]:
summary = []
for year in years: 
    for assay in assays:
        harmonized_assay = '{}_harmonized'.format(assay)
        num_samples = datasets[year][harmonized_assay].subject_id.nunique()
        summary.append([year, assay, num_samples,
                        datasets[year][harmonized_assay].shape[0],
                        datasets[year][harmonized_assay].shape[1]])

In [34]:
summary = pd.DataFrame(summary)
summary.columns = ['Year', 'Assay', 'No. Subjects', 'No. Experiments', 'No. Features']

In [35]:
summary.pivot(index='Year', columns='Assay', values='No. Subjects')

Assay,pbmc_cell_frequency,pbmc_gene_expression,plasma_ab_titer,plasma_cytokine_concentration
Year,,,,
2020,20,36,58,18
2021,33,36,33,36
2022,21,21,20,20


In [36]:
summary.pivot(index='Year', columns='Assay', values='No. Features')

Assay,pbmc_cell_frequency,pbmc_gene_expression,plasma_ab_titer,plasma_cytokine_concentration
Year,,,,
2020,24,11047,17,28
2021,24,11047,17,28
2022,24,11047,17,28
